# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import numpy as np
import pandas as pd
import pickle
import re
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings("ignore")

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

In [2]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer

for i in ("stopwords", "punkt", "wordnet"):
    nltk.download(i)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table("DisasterResponse", engine)
X = df["message"]
Y = df.drop(["id", "message", "original", "genre"], axis=1)

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    # Remove URLs
    url_pattern = "http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"
    url_placeholder = "URLPLACEHOLDER"
    text = re.sub(url_pattern, url_placeholder, text)
    
    # Tokenize
    lemmatizer = WordNetLemmatizer()
    
    tokens = word_tokenize(text)
    tokens = [lemmatizer.lemmatize(t).lower().strip() for t in tokens]
    tokens = [t for t in tokens if t not in tokenize.stopwords]
    return tokens


tokenize.stopwords = set(stopwords.words('english'))

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [38]:
def get_pipeline():
    return Pipeline([
    ('vec', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_estimators=10, max_depth=4, n_jobs=6)))
    ])

pipeline = get_pipeline()

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [39]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
print(len(X_train), len(X_test), len(Y_train), len(Y_test))

19662 6554 19662 6554


In [40]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [41]:
def get_report(pipeline, X, Y):
    YHat = pipeline.predict(X)
    metrics = [
        [
            m(Y.iloc[:, i].values, YHat[:, i], average="micro")
            for m in (f1_score, precision_score, recall_score)
        ]
        for i in range(len(Y.columns))
    ]
    report = pd.DataFrame(metrics, columns=["F1Score", "Precision", "Recall"], index=Y.columns)
    return report


report = get_report(pipeline, X_test, Y_test)
report

,F1Score,Precision,Recall
related,0.763198,0.763198,0.763198
request,0.828959,0.828959,0.828959
offer,0.996186,0.996186,0.996186
aid_related,0.593378,0.593378,0.593378
medical_help,0.918370,0.918370,0.918370
medical_products,0.949649,0.949649,0.949649
search_and_rescue,0.969789,0.969789,0.969789
security,0.983064,0.983064,0.983064
military,0.967958,0.967958,0.967958
child_alone,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [35]:
param_grid = {
    'clf__estimator__n_estimators': [10],
    'clf__estimator__max_depth': [4, 5],
}

cv = GridSearchCV(estimator=pipeline, param_grid=param_grid, cv=5, n_jobs=6)
cv.fit(X_train, Y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vec', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=6,
       param_grid={'clf__estimator__n_estimators': [10], 'clf__estimator__max_depth': [4, 5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [36]:
get_report(cv, X_test, Y_test)

,F1Score,Precision,Recall
related,0.769149,0.769149,0.769149
request,0.822093,0.822093,0.822093
offer,0.994965,0.994965,0.994965
aid_related,0.589869,0.589869,0.589869
medical_help,0.919896,0.919896,0.919896
medical_products,0.949496,0.949496,0.949496
search_and_rescue,0.970247,0.970247,0.970247
security,0.983369,0.983369,0.983369
military,0.969027,0.969027,0.969027
child_alone,1.000000,1.000000,1.000000


In [37]:
cv.best_params_

{'clf__estimator__max_depth': 4, 'clf__estimator__n_estimators': 10}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [42]:
pipeline_alt = Pipeline([
    ('vec', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(AdaBoostClassifier(n_estimators=10)))
])

pipeline_alt.fit(X_train, Y_train)
get_report(pipeline_alt, X_test, Y_test)

,F1Score,Precision,Recall
related,0.763503,0.763503,0.763503
request,0.879463,0.879463,0.879463
offer,0.995880,0.995880,0.995880
aid_related,0.720476,0.720476,0.720476
medical_help,0.925999,0.925999,0.925999
medical_products,0.958193,0.958193,0.958193
search_and_rescue,0.973756,0.973756,0.973756
security,0.981996,0.981996,0.981996
military,0.968569,0.968569,0.968569
child_alone,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [43]:
pickle.dump(pipeline, open("rf_model.pkl", "wb"))
pickle.dump(pipeline_alt, open("alt_model.pkl", "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.